# How to train and load a LDA model using `pke`

A Latent Dirichlet Allocation (LDA) model is required by the TopicalPageRank model for weighting candidates. Below is an example on how to train such model for a given text collection.

In [1]:
!pip install git+https://github.com/boudinfl/pke.git
!pip install datasets
!python -m spacy download en_core_web_sm

  Cloning https://github.com/boudinfl/pke.git to /private/var/folders/_s/dsym612j14gggkqchsd35clh0000gn/T/pip-req-build-7gjb6fw2
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /private/var/folders/_s/dsym612j14gggkqchsd35clh0000gn/T/pip-req-build-7gjb6fw2
  Resolved https://github.com/boudinfl/pke.git to commit dec50293ffd25f63a554e5822af13608686bfc77
  Preparing metadata (setup.py) ... done
  Created wheel for pke: filename=pke-2.0.0-py3-none-any.whl size=6160309 sha256=0fb41ea38a461b3902a6ee84b063e03f9191e99d2fd3c5c4870d2a8b90ae6838
  Stored in directory: /private/var/folders/_s/dsym612j14gggkqchsd35clh0000gn/T/pip-ephem-wheel-cache-vbc3pz88/wheels/8c/07/29/6b35bed2aa36e33d77ff3677eb716965ece4d2e56639ad0aab
Successfully built pke
You should consider upgrading via the '/Users/boudin-f/Documents/GitHub/pke/venv/bin/python -m pip install --upgrade pip' command.


You should consider upgrading via the '/Users/boudin-f/Documents/GitHub/pke/venv/bin/python -m pip install --upgrade pip' command.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.9/13.9 MB 7.9 MB/s eta 0:00:0000:0100:01m


You should consider upgrading via the '/Users/boudin-f/Documents/GitHub/pke/venv/bin/python -m pip install --upgrade pip' command.
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


## Preamble on keyphrase extraction datasets using 🤗 datasets

For simplicity and ease of use, we rely on the `datasets` module from 🤗 huggingface to load and access sample documents from keyphrase extraction datasets. Please have a look at our organization page (https://huggingface.co/taln-ls2n) for more information on which datasets are available.

In [2]:
from datasets import load_dataset

# load the inspec dataset
dataset = load_dataset('taln-ls2n/inspec')

No config specified, defaulting to: inspec/raw
Reusing dataset inspec (/Users/boudin-f/.cache/huggingface/datasets/taln-ls2n___inspec/raw/1.1.0/0ae146cabe770846946b3279b4c751efe0aca2dd68b3f24427d4624cd22bb20d)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import spacy
from tqdm.notebook import tqdm
from spacy.tokenizer import _get_regex_pattern

nlp = spacy.load("en_core_web_sm")

# Tokenization fix for in-word hyphens (e.g. 'non-linear' would be kept 
# as one token instead of default spacy behavior of 'non', '-', 'linear')
# https://spacy.io/usage/linguistic-features#native-tokenizer-additions

from spacy.lang.char_classes import ALPHA, ALPHA_LOWER, ALPHA_UPPER
from spacy.lang.char_classes import CONCAT_QUOTES, LIST_ELLIPSES, LIST_ICONS
from spacy.util import compile_infix_regex

# Modify tokenizer infix patterns
infixes = (
    LIST_ELLIPSES
    + LIST_ICONS
    + [
        r"(?<=[0-9])[+\-\*^](?=[0-9-])",
        r"(?<=[{al}{q}])\.(?=[{au}{q}])".format(
            al=ALPHA_LOWER, au=ALPHA_UPPER, q=CONCAT_QUOTES
        ),
        r"(?<=[{a}]),(?=[{a}])".format(a=ALPHA),
        # ✅ Commented out regex that splits on hyphens between letters:
        # r"(?<=[{a}])(?:{h})(?=[{a}])".format(a=ALPHA, h=HYPHENS),
        r"(?<=[{a}0-9])[:<>=/](?=[{a}])".format(a=ALPHA),
    ]
)

infix_re = compile_infix_regex(infixes)
nlp.tokenizer.infix_finditer = infix_re.finditer

# populates a docs list with spacy doc objects
docs = []
for sample in tqdm(dataset['train']):
    docs.append(nlp(sample["title"]+". "+sample["abstract"]))

  0%|          | 0/1000 [00:00<?, ?it/s]

In [4]:
import logging
from pke import compute_lda_model
from string import punctuation

logging.basicConfig(level=logging.INFO)

compute_lda_model(
    documents=docs,
    output_file="inspec.lda.pickle.gz",
    n_topics=500,               # number of topics
    language='en',              # language of the input files
    stoplist=list(punctuation), # stoplist (punctuation marks)
    normalization='stemming'    # use porter stemmer
)

/usr/local/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
INFO:root:writing LDA model to inspec.lda.pickle.gz


In [5]:
from pke import load_lda_model

lda_model = load_lda_model(input_file="inspec.lda.pickle.gz")

ImportError: cannot import name 'load_lda_model' from 'pke' (/usr/local/lib/python3.9/site-packages/pke/__init__.py)